1. [SparkDataFrames Basics](#1)
2. [Spark Sql](#2)
3. [Spark DataFarme Basic Operations](#3)

# 1
## Spark DataFrame Basics


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
spark

In [4]:
df = spark.read.json('./data/people.json')

In [6]:
#mostrar el dataframe
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
# imprimir el esquema del dataframe
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
# Mostrar las columnas del DataFrame
df.columns

['age', 'name']

In [11]:
# Mostrar el resumen estadístico
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [13]:
# Crear nuestro propio esquema y cambiar los tipos de datos
from pyspark.sql.types import StructField, StringType, IntegerType,StructType

In [14]:
data_schema = [StructField('age',IntegerType(),True),
              StructField('name',StringType(),True)]

In [15]:
final_struc = StructType(fields = data_schema)

In [17]:
df = spark.read.json('./data/people.json',schema = final_struc)

In [20]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [21]:
# Saber el tipo de una columnas
type(df['age'])

pyspark.sql.column.Column

In [22]:
df.select('age')

DataFrame[age: int]

In [23]:
# Mostrar el valor de una columnas
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [27]:
# Añadir un columna al DataFrame.
# Hay que tener en cuenta que no es una operación que modifique el df original.
df.withColumn('newage',df['age']*2).show()


+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    60|
|  19| Justin|    38|
+----+-------+------+



In [28]:
# Cambiar el nombre de una columna
df.withColumnRenamed('age','new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



# 2
## Spark SQL

In [29]:
df.createOrReplaceTempView('people')

In [30]:
results = spark.sql("SELECT * FROM people") 

In [31]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [32]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")

In [33]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



# 3 
## Spark DataFrame Basic Operations

In [37]:
df = spark.read.csv('./data/appl_stock.csv',inferSchema= True,header = True)

In [38]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [39]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

